**Import Libraries**

In [1]:
import pandas as pd


from geopy.geocoders import Nominatim
#import geocoder

ModuleNotFoundError: No module named 'geopy'

**Read Dataset**

In [ ]:
#innovations = pd.read_parquet("geocoding_all_0424.parquet")
innovations = pd.read_parquet("sample_for_geocoding.parquet")

**Preprocessing (minimise time used for geocoding)**

In [ ]:
#drop all duplicates
geocoded_innovations = innovations.drop_duplicates(subset = "user_location", keep ="first")
#drop all rows with lat/lon
geocoded_innovations = geocoded_innovations[geocoded_innovations["longitude"].isna() & geocoded_innovations["latitude"].isna()]
#only "user_location" column
geocoded_innovations = geocoded_innovations[["user_location"]]

**Geocoding**

In [ ]:
geolocator = Nominatim(user_agent="Eric")

geocoded_innovations["geocoded"] = geocoded_innovations.user_location.apply(geolocator.geocode)

In [ ]:
#Store lat and lon in separate columns 
geocoded_innovations['Lat'] = geocoded_innovations["geocoded"].apply(lambda x: x.latitude if x else None)
geocoded_innovations['Lon'] = geocoded_innovations["geocoded"].apply(lambda x: x.longitude if x else None)

**Afterprocessing**

In [ ]:
#drop "user_location" column before joining two dataset (dropping column, which can be found in both datasets (apart from index))
geocoded_innovations.drop("user_location", axis=1, inplace = True)

#joining two dataframes (left join full dataset and geocoded subset)
result = innovations.join(geocoded_innovations)

#Populate NaN values in columns "geocoded", "Lat" and "Lon" of same locations
result["geocoded"] = (result["geocoded"].fillna(result.groupby("user_location")["geocoded"].transform("first")))
result["Lat"] = (result["Lat"].fillna(result.groupby("user_location")["Lat"].transform("first")))
result["Lon"] = (result["Lon"].fillna(result.groupby("user_location")["Lon"].transform("first")))

result



In [ ]:
result

**Interactive Visualisation**

In [ ]:
import folium
from folium.plugins import HeatMapWithTime
import numpy as np
from numpy import nan
import math

In [ ]:
#Removing all row, where Lat column has NaN values
nonan = result[result["Lat"].notna()]

In [ ]:
lat_lon_list = []
for i in nonan["created_at"].unique():
    temp=[]
    for index, instance in nonan[nonan["created_at"] == i].iterrows():
        temp.append([instance["Lat"],instance["Lon"]])
    lat_lon_list.append(temp)


In [ ]:
#converting it to datetime format
nonan["created_at"]= pd.to_datetime(nonan["created_at"])#creating a time index
time_index = []
for i in nonan["created_at"].unique():
    time_index.append(i)#formatting the index
date_strings = [d.strftime('%d/%m/%Y, %H:%M:%S') for d in time_index]

In [ ]:
m = folium.Map([47.32, 8.50], zoom_start=5, tiles = "CartoDB dark_matter", control_scale = True)

HeatMapWithTime(lat_lon_list,radius=5,auto_play=True,position='bottomright',name="cluster",index=date_strings,max_opacity=1).add_to(m)

m

In [ ]:
# Todo
# Geocode all dataframe
# add innovation selector to folium
# find better visualisation in folium. not heatmap but point
# Move coordinates for origninally geocoded tweets from columns "Latitude" and "Longitude" to "Lat" and "Lon"